In [11]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
from image_processor import transformation
from classifier.models import CNN
from classifier.Main import CNN_prediction


In [2]:
import os
os.getcwd()

'/mnt/c/Users/PC-BARRYZHANG/Dropbox/Homework/EE475/EE475-Final-Project'

In [4]:
x_train = np.array(transformation("thicccc.jpg"))
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_train = np.expand_dims(x_train, -1)
# Make sure images have shape (28, 28, 1)

184 909


In [5]:
cnn = CNN()
cnn.load_weights('weights/ckpt_CNN')

## CNN_prediction(x_train)

In [6]:
probs = cnn.predict(x_train)
np.argmax(probs, axis=1)

array([9, 2, 8, 6, 4, 5, 3, 8, 2, 1])

In [9]:
show_images(x_train, y_test, 10)

NameError: name 'y_test' is not defined

#### Load data

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print("Number of original training examples:", len(x_train))
print("Number of original test examples:", len(x_test))

In [ ]:
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train_), (x_test, y_test_) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train_, num_classes)
y_test = keras.utils.to_categorical(y_test_, num_classes)

In [ ]:
INPUT_SHAPE = (28,28,1)
NUM_CLASSES = 10

class CNN(keras.Model):

    def __init__(self):
        super(CNN, self).__init__()
        self.model = keras.models.Sequential()
        self.model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=1, padding='Same', activation='relu',
                                      input_shape = INPUT_SHAPE) )
        self.model.add(keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'))
        self.model.add(keras.layers.Conv2D(filters=64,kernel_size=(3, 3),strides=1,padding='same', activation='relu'))
        self.model.add(keras.layers.MaxPool2D(pool_size=(2, 2),padding='same'))
        self.model.add(keras.layers.Flatten())
        self.model.add(keras.layers.Dense(units=NUM_CLASSES, activation='sigmoid'))

    def call(self, inputs):
        return self.model(inputs)

class CNN_Drop(keras.Model):

    def __init__(self):
        super(CNN_Drop, self).__init__()
        self.model = keras.models.Sequential()
        self.model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=1, padding='Same', activation='relu',
                                      input_shape = INPUT_SHAPE) )
        self.model.add(keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'))
        self.model.add(keras.layers.Conv2D(filters=64,kernel_size=(3, 3),strides=1,padding='same', activation='relu'))
        self.model.add(keras.layers.MaxPool2D(pool_size=(2, 2),padding='same'))
        self.model.add(keras.layers.Flatten())
        self.model.add(keras.layers.Dropout(0.5))
        self.model.add(keras.layers.Dense(units=NUM_CLASSES, activation='sigmoid'))

    def call(self, inputs):
        return self.model(inputs)

class CNN_BN(keras.Model):

    def __init__(self):
        super(CNN_BN, self).__init__()
        self.model = keras.models.Sequential()
        self.model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), strides=1, padding='Same', input_shape = INPUT_SHAPE) )
        self.model.add(keras.layers.BatchNormalization())
        self.model.add(keras.layers.ReLU())
        self.model.add(keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'))
        
        self.model.add(keras.layers.Conv2D(filters=64,kernel_size=(3, 3),strides=1,padding='same'))
        self.model.add(keras.layers.BatchNormalization())
        self.model.add(keras.layers.ReLU())
        self.model.add(keras.layers.MaxPool2D(pool_size=(2, 2),padding='same'))
        
        self.model.add(keras.layers.Flatten())
        self.model.add(keras.layers.Dense(units=NUM_CLASSES, activation='sigmoid'))

    def call(self, inputs):
        return self.model(inputs)


#### CNN Model

In [ ]:
epochs = 20
batch_size = 128
cnn = CNN()
cnn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

CNN_history = cnn.fit(x_train[:3000], y_train[:3000], batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
score = cnn.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
cnn.save_weights("ckpt_CNN")

In [ ]:
new_cnn = CNN()
new_cnn.load_weights("ckpt_CNN")
cnn.evaluate(x_test, y_test, verbose=0)

#### CNN_Drop

In [ ]:
epochs = 20
batch_size = 264
cnn_drop = CNN_Drop()
cnn_drop.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

CNN_drop_history = cnn_drop.fit(x_train[:3000], y_train[:3000], batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
score = cnn_drop.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
cnn_drop.save_weights("ckpt_CNN_Drop")

#### CNN_BN

In [ ]:
epochs = 20
batch_size = 128
cnn_bn = CNN_BN()
cnn_bn.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

CNN_BN_history = cnn_bn.fit(x_train[:3000], y_train[:3000], batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
score = cnn_bn.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

In [ ]:
cnn_bn.save_weights("ckpt_CNN_BN")

In [ ]:
np.argmax(cnn_bn.predict(x_test[:3]), axis=1)

In [ ]:
plt.imshow(x_test[0])

#### Comparison

Plot

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

def plot_losses(histories, titles,):
    n = len(histories)
    fig, axes = plt.subplots(1,n, figsize = (n*6,4))
    if n == 1:
        axes = [axes]

    for i in range(n):
        train_loss = histories[i].history['loss']
        val_loss = histories[i].history['val_loss']
        x = np.arange(len(train_loss),)
        axes[i].plot(x, train_loss,color='blue', label = 'Train loss')
        axes[i].plot(x, val_loss,color='red', label = 'Validation loss')
        axes[i].set_xlabel('Epoch')
        axes[i].set_ylabel('Loss')
        axes[i].set_title(titles[i])
        axes[i].legend()
        axes[i].xaxis.set_major_locator(MaxNLocator(integer=True))
    
    plt.show()

In [ ]:
plot_losses([CNN_history, CNN_drop_history, CNN_BN_history ], ['CNN', 'CNN Dropout', 'CNN BatchNorm'])
# plot_losses([CNN_history], ['CNN'])

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

def plot_accuracy(histories, titles,):
    n = len(histories)
    fig, axes = plt.subplots(1,n, figsize = (n*6,4))
    if n == 1:
        axes = [axes]

    for i in range(n):
        train_loss = histories[i].history['accuracy']
        val_loss = histories[i].history['val_accuracy']
        x = np.arange(len(train_loss),)
        axes[i].plot(x, train_loss, color='blue', label = 'Train accuracy')
        axes[i].plot(x, val_loss, color ='red', label = 'Validation accuracy')
        axes[i].set_xlabel('Epoch')
        axes[i].set_ylabel('Loss')
        axes[i].set_title(titles[i])
        axes[i].legend()
        axes[i].xaxis.set_major_locator(MaxNLocator(integer=True))

    plt.show()

In [ ]:
plot_accuracy([CNN_history, CNN_drop_history, CNN_BN_history], ['CNN', 'CNN Dropout', 'CNN BatchNorm'])
# plot_accuracy([CNN_history], ['CNN'])

In [8]:
def show_images(x,y, num):
    if num % 2 == 1:
        num += 1
    
    images = x_train.squeeze()[:num]
    labels = y_train.squeeze()[:num]
    num_row = 2
    num_col = num // 2

    fig, axes = plt.subplots(num_row, num_col, figsize=(1.5*num_col,2*num_row))
    for i in range(num):
        ax = axes[i//num_col, i%num_col]
        ax.imshow(images[i], cmap='gray_r', )
        ax.set_title('Label: {}'.format(labels[i]))

    plt.tight_layout()
    plt.show()

In [ ]:
show_images(x_train, y_test, 10)